## Importing libraries

In [ ]:
import numpy as np
import pandas as pd

## Making new directory

In [ ]:
!mkdir fashion_mnist_test
!mkdir fashion_mnist_train

## Unziping the data

In [ ]:
!unzip test.zip -d fashion_mnist_test

In [ ]:
!unzip train.zip -d fashion_mnist_train

## Images in the train and test set 

In [ ]:
import os

print(len(os.listdir('/content/fashion_mnist_train/train')))
print(len(os.listdir('/content/fashion_mnist_test/test')))

60000
10000


In [ ]:
train=pd.read_csv('/content/train.csv')
test=pd.read_csv('/content/test.csv')


In [ ]:
train.head()

,id,label
0,1,9
1,2,0
2,3,0
3,4,3
4,5,0


In [ ]:
test.head()

,id
0,60001
1,60002
2,60003
3,60004
4,60005


In [ ]:
train.label.unique()

array([9, 0, 3, 2, 7, 5, 1, 6, 4, 8])

In [ ]:
train.id=train.id.astype(str)
train.label=train.label.astype(str)
test.id=test.id.astype(str)

In [ ]:
for i in range(len(train)):
  train.iloc[i,0]=train.iloc[i,0]+'.png'

In [ ]:
for i in range(len(test)):
  test.iloc[i,0]=test.iloc[i,0]+'.png'

In [ ]:
train=train.rename(columns={'id':'filename'})

In [ ]:
train.head()

,filename,label
0,1.png,9
1,2.png,0
2,3.png,0
3,4.png,3
4,5.png,0


In [ ]:
test=test.rename(columns={'id':'filename'})

In [ ]:
test.head()

,filename
0,60001.png
1,60002.png
2,60003.png
3,60004.png
4,60005.png


## Using data flow from dataframe and model builidng

In [ ]:
# flow from dataframe

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten,Conv2D,MaxPooling2D
from keras.models import Sequential


train_datagen=ImageDataGenerator(rescale=1.0/255.0,validation_split=0.20)
test_datagen=ImageDataGenerator(rescale=1.0/255.0)

train_generator= train_datagen.flow_from_dataframe(dataframe=train,directory='/content/fashion_mnist_train/train',
                                                   x_col='filename',
                                                   y_col='label',
                                                   seed=7,
                                                   shuffle=True,
                                                   subset='training',
                                                   class_mode='categorical',
                                                   target_size=(28,28),batch_size=32)

test_generator= test_datagen.flow_from_dataframe(dataframe=test,directory='/content/fashion_mnist_test/test',
                                                   x_col='filename',
                                                   y_col='label',
                                                   seed=7,
                                                   shuffle=False,
                                                   class_mode=None,
                                                   target_size=(28,28),batch_size=32,)

val_generator= train_datagen.flow_from_dataframe(dataframe=train,directory='/content/fashion_mnist_train/train',
                                                   x_col='filename',
                                                   y_col='label',
                                                   seed=7,
                                                   shuffle=True,
                                                   subset='validation',
                                                   class_mode='categorical',
                                                   target_size=(28,28),batch_size=32)


model=Sequential([Conv2D(32,(3,3),activation='relu',input_shape=(28,28,3),padding='same'),
                  MaxPooling2D(),
                  Conv2D(32,(3,3),activation='relu',padding='same'),
                  MaxPooling2D(),
                  Flatten(),
                  Dense(1000,activation='relu'),
                  Dense(512,activation='relu'),
                  Dense(10,activation='softmax')])
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()


Found 48000 validated image filenames belonging to 10 classes.
Found 10000 validated image filenames.
Found 12000 validated image filenames belonging to 10 classes.
Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 28, 28, 32)        896       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 14, 14, 32)        9248      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 7, 7, 32)          0         
_________________________________________________________________
flatten_14 (Flatten)         (None, 1568)              0         
_________________________________________________________________
dense_42 (Dense)    

## Training the model

In [ ]:
history=model.fit(train_generator,epochs=10,verbose=1,validation_data=val_generator)

Epoch 1/10
1500/1500 [==============================] - 17s 11ms/step - loss: 0.3906 - accuracy: 0.8570 - val_loss: 0.2803 - val_accuracy: 0.8937
Epoch 2/10
1500/1500 [==============================] - 17s 11ms/step - loss: 0.2614 - accuracy: 0.9024 - val_loss: 0.2779 - val_accuracy: 0.8987
Epoch 3/10
1500/1500 [==============================] - 17s 11ms/step - loss: 0.2162 - accuracy: 0.9184 - val_loss: 0.2371 - val_accuracy: 0.9118
Epoch 4/10
1500/1500 [==============================] - 17s 12ms/step - loss: 0.1810 - accuracy: 0.9314 - val_loss: 0.2289 - val_accuracy: 0.9197
Epoch 5/10
1500/1500 [==============================] - 18s 12ms/step - loss: 0.1476 - accuracy: 0.9454 - val_loss: 0.2388 - val_accuracy: 0.9206
Epoch 6/10
1500/1500 [==============================] - 18s 12ms/step - loss: 0.1225 - accuracy: 0.9532 - val_loss: 0.2796 - val_accuracy: 0.9202
Epoch 7/10
1500/1500 [==============================] - 17s 12ms/step - loss: 0.1008 - accuracy: 0.9621 - val_loss: 0.2695 -

## Evaluation on validation data

In [ ]:
model.evaluate_generator(val_generator)

[0.3852987587451935, 0.9181666374206543]

## Predictions on test images

In [ ]:
prediction=model.predict_generator(test_generator)
prediction

Instructions for updating:
Please use Model.predict, which supports generators.


array([[1.6653150e-21, 1.8791116e-21, 1.7123535e-22, ..., 4.9344074e-13,
        4.4660059e-22, 1.0000000e+00],
       [3.3278624e-08, 1.0610140e-13, 9.9999523e-01, ..., 5.9507817e-15,
        1.1014708e-11, 4.0496769e-18],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [8.4413848e-26, 0.0000000e+00, 1.2963549e-29, ..., 8.5766874e-30,
        1.0000000e+00, 6.0690822e-30],
       [0.0000000e+00, 1.0000000e+00, 2.6194371e-37, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [5.1301154e-03, 8.2443499e-08, 9.4083342e-04, ..., 5.2573609e-01,
        3.1395859e-01, 8.5235788e-03]], dtype=float32)

In [ ]:
prediction.shape

(10000, 10)

In [ ]:
pred=[]
for i in range(0,10000):
  pred.append(np.argmax(prediction[i]))
  

In [ ]:
sample_submission=pd.read_csv('/content/sample_submission.csv')
sample_submission.head()

,id,label
0,60001,0
1,60002,0
2,60003,0
3,60004,0
4,60005,0


In [ ]:
submission=pd.DataFrame()
submission['id']=sample_submission['id']
submission['label']=pred


In [ ]:
submission.head()

,id,label
0,60001,9
1,60002,2
2,60003,1
3,60004,1
4,60005,6


In [ ]:
submission.to_csv('/content/submission.csv',index=False)